### Evaluate options for graph embeddings 

In [1]:
import pickle
import networkx as nx

g_path = "/home/tmuehlen/repos/graph_coverage/actor_graphs/carla/graph_2025-08-23 23:19:38.811319_0_0.pkl"
with open(g_path, "rb") as file:
    graph = pickle.load(file)



In [2]:
type(graph)

networkx.classes.multidigraph.MultiDiGraph

In [6]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import from_networkx
import networkx as nx
import numpy as np

# This is the GNN architecture you provided
class GCNForGraphEmbedding(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GCNForGraphEmbedding, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 1) # Example for a regression task

    def forward(self, x, edge_index, batch):
        # 1. GNN layers to get node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Global Pooling layer to get graph embedding
        graph_embedding = global_mean_pool(x, batch)

        # 3. MLP head for final task
        x = F.dropout(graph_embedding, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x, graph_embedding



In [7]:
# --- Step 1: Create an example NetworkX graph ---
G = nx.erdos_renyi_graph(n=15, p=0.3) # 15 nodes, random connections

# GNNs need node features. Let's add some random features to each node.
# Let's say each node has 5 features.
for node in G.nodes():
    G.nodes[node]['x'] = np.random.rand(5).tolist() # 'x' is the standard attribute name for features in PyG

# --- Step 2: Convert the NetworkX graph to a PyG Data object ---
# The from_networkx utility handles the conversion automatically.
# It looks for node attributes named 'x' for features and edge attributes named 'edge_attr'.
data = from_networkx(G, group_node_attrs=['x'])

In [8]:


print("PyG Data object:")
print(data)
print("="*30)

PyG Data object:
Data(edge_index=[2, 62], x=[15, 5])


In [9]:

# --- Step 3: Instantiate the model and run the forward pass ---

# Define model parameters
num_node_features = data.num_node_features
hidden_channels = 64

# Instantiate the model
model = GCNForGraphEmbedding(num_node_features=num_node_features, 
                             hidden_channels=hidden_channels)
model.eval() # Set model to evaluation mode


GCNForGraphEmbedding(
  (conv1): GCNConv(5, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=1, bias=True)
)

In [10]:


# The 'batch' vector is needed for pooling. For a single graph, it's a tensor of zeros.
# It tells the pooling layer that all nodes belong to the same graph.
batch = torch.zeros(data.num_nodes, dtype=torch.long)

# Get the model output
final_output, graph_embedding = model(data.x, data.edge_index, batch)

print(f"Final Prediction (from MLP head): {final_output.item()}")
print(f"Shape of the Graph Embedding: {graph_embedding.shape}")
print("Graph Embedding Vector:")
print(graph_embedding)

Final Prediction (from MLP head): -0.023900717496871948
Shape of the Graph Embedding: torch.Size([1, 64])
Graph Embedding Vector:
tensor([[-0.2151, -0.0991,  0.0992, -0.1266, -0.0637, -0.0287, -0.1408, -0.1387,
         -0.0736,  0.0758,  0.0057,  0.0289,  0.0157, -0.1015, -0.0241,  0.0422,
          0.0346, -0.0517, -0.0959,  0.0200, -0.0303,  0.0510,  0.0958, -0.0030,
         -0.0386, -0.0098,  0.0141, -0.0939,  0.0014, -0.0446, -0.0390,  0.1108,
         -0.0240, -0.1473,  0.0036, -0.0178, -0.0837, -0.0904,  0.1827, -0.0775,
          0.0062, -0.0391, -0.0061,  0.1616, -0.0870,  0.0120, -0.0993, -0.0754,
         -0.0581,  0.0055, -0.0072,  0.1329, -0.0029, -0.1162,  0.0278,  0.0835,
         -0.1292, -0.0088,  0.0805, -0.0362,  0.0530,  0.0900, -0.1261, -0.2070]],
       grad_fn=<DivBackward0>)


In [5]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GCNForGraphEmbedding(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GCNForGraphEmbedding, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 1) # Example for a regression task

    def forward(self, x, edge_index, batch):
        # 1. GNN layers to get node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Global Pooling layer to get graph embedding
        graph_embedding = global_mean_pool(x, batch)

        # 3. MLP head for final task
        x = F.dropout(graph_embedding, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x, graph_embedding